In [ ]:
import os
import plotly.express as px
import numpy as np
import boto3
import json
import pandas as pd

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

from mirrorverse.utils import read_data_w_cache

In [ ]:
data = read_data_w_cache(
    'select * from chinook_depth_experiment_3_3_26'
)
print(data.shape)
data.head()

In [ ]:
bri = data[data['epoch'] == 75].sort_values('val_loss', ascending=True)['run_id'].values[0]
bri

In [ ]:
data[data['epoch'] == 75].sort_values('val_loss', ascending=True)['val_loss'].values[0]

In [ ]:
px.line(data[data['epoch'] <= 75].sort_values(['run_id', 'epoch']), x='epoch', y='train_loss', color='run_id')

In [ ]:
px.line(data[data['epoch'] <= 75].sort_values(['run_id', 'epoch']), x='epoch', y='val_loss', color='run_id')

In [ ]:
s3_client = boto3.client('s3')
bucket = 'mimic-log-odds-models'
prefix = 'chinook-depth-3-1-18'

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

configs = {}
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith('config.json'):
            config = json.loads(s3_client.get_object(Bucket=bucket, Key=obj['Key'])['Body'].read().decode('utf-8'))
            if 'model' in config:
                configs[config['run_id']] = config['model']
else:
    print("No objects found.")

In [ ]:
configs[bri]

In [ ]:
rows = []
for run_id, config in configs.items():
    row = {
        key: value
        for key, value in config.items()
        if key not in ['optimizer_kwargs', 'layers']
    }
    row['run_id'] = run_id
    rows.append(row)
cdf = pd.DataFrame(rows).merge(
    data[data['epoch'] == 75], on='run_id', how='inner'
)
cdf.head()

In [ ]:
px.scatter(
    cdf, x='layer_size', y='val_loss', color='num_layers',
    facet_col='batch_size'
)

In [ ]:
sql = '''
select 
    *
from 
    mgietzmann_tag_depths
'''

data = read_data_w_cache(
    sql
)
print(data.shape)
data.head()

In [ ]:
data['time'] = pd.to_datetime(data['epoch'], unit='s')
data['depth'] = -data['depth']
px.line(
    data[data['tag_key'] == '129843'].sort_values('epoch'), x='time', y='depth'
)

In [ ]:
px.line(
    data[data['tag_key'] == '129843'].sort_values('epoch'), x='time', y='depth'
)

In [ ]:
data['tag_key'].unique()

In [ ]:
data[data['depth'] == data['depth'].min()]